In [4]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
import riskfolio as rp
import warnings
import orjson as json
warnings.filterwarnings("ignore")

In [ ]:
with open('passed.txt') as f:
    tickers = f.read().splitlines()
tickers.append('^TNX')  # 10 year treasury yield
with open('ticker_capm.json') as f:
    ticker_capm = json.loads(f.read())
ticker_info = pd.read_csv('yahoo_tickers.csv')
MAR = 0.13256  # see calc_r.txt

In [ ]:
def download_data_and_calc_returns(tickers, period='5y', interval='1d', prepost=False):
    if isinstance(tickers, list):
        data = yf.download(tickers, start='2017-11-30', end='2022-11-30', interval=interval, prepost=prepost, threads=True)
        data = data.loc[:,('Adj Close', slice(None))]  # get adjusted close only
        data.columns = tickers
        data['WYN.L']['2022-06-14'] = 628.0
        data = data[tickers].pct_change().dropna()
        return data
    else:
        data = yf.download(tickers, period=period, interval=interval, prepost=prepost, threads=True)
        data = data.loc[:, 'Adj Close']
        data = data.pct_change().dropna()
        return data

returns = download_data_and_calc_returns(tickers)
returns.index = returns.index.map(lambda a: pd.to_datetime(a).date())  # remove timezone awareness by converting to dates only
returns = returns.sum(level=0)  # merge 2 days into 1 since different market has diff closing time

In [ ]:
nasdaq_returns = download_data_and_calc_returns('^GSPC', period='10y')
nasdaq_slpm = rp.RiskFunctions.LPM(nasdaq_returns, MAR=MAR/252, p=2)
nasdaq_slpm

In [ ]:
port = rp.Portfolio(returns=returns, sht=False, lowerret=MAR/252, upperslpm=nasdaq_slpm)

In [ ]:
method_to_estimate_expected_return = 'ewma1'  # useless, will be overridden with custom returns
method_to_estimate_covariance = 'hist'
port.assets_stats(method_mu=method_to_estimate_expected_return, method_cov=method_to_estimate_covariance, d=0.99)  # lower d means place more emphasis on recent performance, 0 <= d < 1

In [ ]:
port.mu['^TNX'] = 0.04/252  # set risk free asset

In [ ]:
def custom_returns(col):
    if col.name != '^TNX':
        return pd.Series([ticker_capm[col.name] / (252*100)])
    else:
        return col
port.mu = port.mu.apply(custom_returns, axis=0)
port.mu*252

In [ ]:
assets = port.mu.columns.values.tolist()

model = 'Classic'  # based on history
risk_measure = 'SLPM'  # Sortino Ratio
obj = 'MinRisk'
risk_free_rate = MAR/252  # MAR converted to daily

#w = port.optimization(model=model, rm=risk_measure, obj=obj, rf=risk_free_rate)
with open('pp.txt') as f:
    weights = f.read().splitlines()
weights = list(map(lambda x: float(x), weights))
weights = [w*1/np.sum(weights) for w in weights]
base = {'tick': assets, 'weights': weights}
w = pd.DataFrame(base).set_index('tick')
ax = rp.plot_pie(w=w, title=f'{risk_measure} {obj}', others=0.01, nrow=len(port.assetslist), cmap="tab20", height=6, width=10, ax=None)

# Marked as markdown as code below are useless
multiplier = [1 for i in range(63)]

#multiplier[port.assetslist.index('CIPLA.BO')] = 0.6
#multiplier[port.assetslist.index('ECLERX.BO')] = 0.5
#multiplier[port.assetslist.index('INFY.BO')] = 0.5
multiplier[port.assetslist.index('NVDA')] = 1.25
multiplier[port.assetslist.index('JPM')] = 1.25
#multiplier[port.assetslist.index('WMT')] = 0.8
#multiplier[port.assetslist.index('MET')] = 0.7
#multiplier[port.assetslist.index('HSBA.L')] = 0.9
#multiplier[port.assetslist.index('BBSE3.SA')] = 0.5


#multiplier[port.assetslist.index('AMZN')] = 1.1
multiplier[port.assetslist.index('TR')] = 0.7
multiplier[port.assetslist.index('NTES')] = 0.5
#multiplier[port.assetslist.index('TCS.BO')] = 0.5
#multiplier[port.assetslist.index('FFH.TO')] = 0.5
#multiplier[port.assetslist.index('SDR.L')] = 0.5
#multiplier[port.assetslist.index('LMT')] = 0.5
#multiplier[port.assetslist.index('WYN.L')] = 0.5

print(multiplier)
port.mu = np.multiply(port.mu, multiplier)
port.mu*252

In [ ]:
asset_classes = {'Assets': assets,
                 'Industry': [ticker_info.loc[ticker_info['Ticker'] == ticker]['GICS Sector'].to_list()[0] for ticker in assets[:-1]]+['Treasury'],
                 'Country': [ticker_info.loc[ticker_info['Ticker'] == ticker]['Country'].to_list()[0] for ticker in assets[:-1]]+['NA'],
                 'n': ['Yes' if asset == 'NVDA' else 'No' for asset in assets],
                 'c': ['Yes' if asset == 'CIPLA.BO' else 'No' for asset in assets]}
asset_classes = pd.DataFrame(asset_classes)
asset_classes = asset_classes.sort_values(by=['Assets'])
asset_classes

In [ ]:
views = pd.read_csv('views.csv')
views.fillna('', inplace=True)
views

In [ ]:
P, Q = rp.assets_views(views, asset_classes)
pd.DataFrame(P.T)

In [ ]:
port.blacklitterman_stats(P, Q/252, rf=risk_free_rate, w=w, delta=None, eq=True)
model='BL'
rm = 'SLPM'
obj = 'MinRisk'
hist = True
w_bl = port.optimization(model=model, rm=rm, obj=obj, rf=risk_free_rate, hist=hist)

def swap_rows(df, i1, i2):
    a, b = df.loc[i1].copy(), df.loc[i2].copy()
    df.loc[i1], df.loc[i2] = b, a
    return df
swap_rows(w_bl, 'WYN.L','NVDA')
swap_rows(w_bl, 'CIPLA.BO','NVDA')
swap_rows(w_bl, 'JPM','FCN')
swap_rows(w_bl, 'WYN.L','WMT')
swap_rows(w_bl, 'TR','FCN')
swap_rows(w_bl, '600519.SS','COST')
w_bl.T

In [ ]:
ax = rp.plot_pie(w=w_bl, title=f'{risk_measure} {obj}', others=0.0001, nrow=len(port.assetslist), cmap="tab20", height=6, width=10, ax=None)

In [ ]:
returns.index = returns.index.map(lambda a: pd.to_datetime(a).date())  # remove timezone awareness by converting to dates only
rp.Reports.excel_report(returns, w_bl, rf=risk_free_rate, alpha=0.05, t_factor=252, ini_days=1, days_per_year=252, name='portfolio_report')
rp.Reports.jupyter_report(returns, w_bl, rm=risk_measure, rf=risk_free_rate, alpha=0.05, others=0.00001, nrow=len(port.assetslist), height=6, width=14, t_factor=252, ini_days=1, days_per_year=252, bins=100)